### Part2-4 Getting supermarket information from Foursquare　 

In [1]:
import pandas as pd

import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [2]:
df0 = pd.read_csv('listings.csv',encoding='UTF-8')

In [3]:
df0.shape

(37012, 74)

#### Limit the data to properties that meet the following criteria
- Property_type: Entire apartment or Entire condominium or Entire serviced apartment
- review_scores_rating >= 95
- maximum_nights >= 90


In [4]:
df0= df0[(df0['review_scores_rating'] >= 95) & (df0['maximum_nights'] >= 90)]

In [5]:
df0.shape

(12043, 74)

In [6]:
list = ['Entire apartment','Entire condominium','Entire serviced apartment']
df = df0[df0.property_type.isin(list)]

In [7]:
df.shape

(6036, 74)

In [8]:
dfs = pd.read_csv('station.csv')

In [9]:
dfs.columns

Index(['Unnamed: 0', 'Station_Name', 'LINE', 'lat', 'lng', 'numuni'], dtype='object')

In [10]:
dfs['num_super'] = 0


In [11]:
s_columns =['referralId', 'reasons.count', 'reasons.items', 'venue.id',\
       'venue.name', 'venue.location.address', 'venue.location.lat',\
       'venue.location.lng', 'venue.location.labeledLatLngs',\
       'venue.location.distance', 'venue.location.postalCode',\
       'venue.location.cc', 'venue.location.city', 'venue.location.state',\
       'venue.location.country', 'venue.location.formattedAddress',\
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',\
       'venue.location.crossStreet', 'venue.delivery.id', 'venue.delivery.url',\
       'venue.delivery.provider.name', 'venue.delivery.provider.icon.prefix',\
       'venue.delivery.provider.icon.sizes',\
       'venue.delivery.provider.icon.name', 'venue.location.neighborhood',\
       'venue.venuePage.id', 'Station', 'LINE']
super_venues = pd.DataFrame(columns=s_columns)
super_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name,venue.location.neighborhood,venue.venuePage.id,Station,LINE


In [12]:
CLIENT_ID = '(Dummy)'
CLIENT_SECRET = '(Dummy)'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: (Dummy)
CLIENT_SECRET:(Dummy)


##### Category ID for Supermarket is  52f2ab2ebcbc57f1066b8b46

In [13]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

#
for k in range(len(dfs)):
   n_latitude  =  dfs['lat'][k]
   n_longitude =  dfs['lng'][k]
   Station_N = dfs['Station_Name'][k]
   Line_N = dfs['LINE'][k]

# create URL
   url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId=52f2ab2ebcbc57f1066b8b46&limit={}'.format(
      CLIENT_ID, 
      CLIENT_SECRET, 
      VERSION, 
      n_latitude, 
      n_longitude, 
      radius, 
      LIMIT)
# 
#url # display URL

   results = requests.get(url).json()
   venues = results['response']['groups'][0]['items']
   nearby_venues = json_normalize(venues) # flatten JSON
   nearby_venues['Station'] = Station_N
   nearby_venues['LINE'] = Line_N
   super_venues = pd.concat([super_venues,nearby_venues])
   num_sup = len(nearby_venues)
   print(k,Station_N,num_sup) 
   dfs.iloc[k,6] = num_sup


<ipython-input-167-31ccace10a75>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


0 Astor Pl 4
1 Canal St 8
2 50th St 4
3 Bergen St 2
4 Pennsylvania Ave 3
5 238th St 2
6 Cathedral Pkwy (110th St) 6
7 Kingston - Throop Aves 2
8 65th St 4
9 36th St 0
10 Delancey St - Essex St 6
11 Van Siclen Ave 2
12 Norwood Ave 1
13 104th-102nd Sts 3
14 DeKalb Ave 2
15 Beach 105th St 1
16 Beach 90th St 2
17 Freeman St 1
18 Intervale Ave 0
19 182nd-183rd Sts 3
20 174th-175th Sts 5
21 167th St 3
22 Mets - Willets Point 0
23 Junction Blvd 2
24 Flushing - Main St 14
25 Buhre Ave 1
26 3rd Ave - 138th St 2
27 Castle Hill Ave 3
28 Brooklyn Bridge - City Hall 3
29 Zerega Ave 2
30 Grand Central - 42nd St 4
31 33rd St 0
32 96th St 5
33 77th St 4
34 Chauncey St 1
35 Union St 2
36 Elmhurst Ave 8
37 Ralph Ave 1
38 Pelham Pkwy 1
39 Gun Hill Rd 1
40 Nereid Ave (238 St) 2
41 Franklin Ave 3
42 Simpson St 2
43 Bronx Park East 2
44 Winthrop St 1
45 Van Siclen Ave 2
46 149th St - Grand Concourse 1
47 161st St - Yankee Stadium 2
48 Lexington Ave - 59th St 4
49 E 149th St 0
50 Morrison Av - Soundview 2
51

406 183rd St 3
407 Fordham Rd 4
408 World Trade Center 3
409 Canal St - Holland Tunnel 2
410 155th St 2
411 163rd St - Amsterdam Av 5
412 Fulton St 7
413 Chambers St 3
414 Canal St 8
415 City Hall 1
416 Canal St 8
417 South Ferry 0
418 Bowling Green 2
419 Wall St 7
420 Whitehall St 1
421 Rector St 4
422 Fresh Pond Rd 2
423 Middle Village - Metropolitan Ave 0
424 Rector St 4
425 Cortlandt St 5
426 Fulton St 7
427 Broad St 6
428 Cortlandt St 5
429 Wall St 7
430 Dyckman St 3
431 Grand St 11
432 Broadway - Lafayette St 3
433 Bowery 10
434 Canal St 8
435 23rd St 4
436 34th St - Penn Station 0
437 Jackson Hts - Roosevelt Av 8
438 14th St 3
439 135th St 1
440 14th St 3
441 6th Ave 1
442 8th Ave 3
443 14th St 3
444 Nostrand Ave 3
445 Clark St 2
446 Franklin Ave 3
447 Clinton - Washington Aves 3
448 Forest Ave 2
449 110th St 5
450 86th St 5
451 York St 3
452 High St 2
453 Lafayette Ave 2
454 President St 4
455 Woodlawn 1
456 Bleecker St 3
457 103rd St 5
458 Euclid Ave 3
459 88th St 0
460 Cortel

In [14]:
super_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name,venue.location.neighborhood,venue.venuePage.id,Station,LINE
0,e-0-4a98599ef964a520a62b20e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4a98599ef964a520a62b20e3,Sunrise Mart,4 Stuyvesant St,40.730150,-73.989448,"[{'label': 'display', 'lat': 40.73014952006944...",137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95825975,Astor Pl,4-6-6 Express
1,e-0-441a8599f964a52036311fe3-1,0,"[{'summary': 'This spot is popular', 'type': '...",441a8599f964a52036311fe3,Trader Joe's,142 E 14th St,40.733515,-73.987722,"[{'label': 'display', 'lat': 40.73351486923294...",477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Astor Pl,4-6-6 Express
2,e-0-4b8d2319f964a520c8e932e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b8d2319f964a520c8e932e3,Gristedes Supermarkets,25 University Pl,40.731977,-73.994553,"[{'label': 'display', 'lat': 40.73197675660857...",363,...,1007888,https://www.seamless.com/menu/gristedes-superm...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN,Astor Pl,4-6-6 Express
3,e-0-45b3d761f964a520b1411fe3-3,0,"[{'summary': 'This spot is popular', 'type': '...",45b3d761f964a520b1411fe3,Food Emporium,10 Union Sq E,40.734341,-73.989531,"[{'label': 'display', 'lat': 40.73434079522086...",494,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Astor Pl,4-6-6 Express
0,e-0-4b5cae76f964a520203e29e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b5cae76f964a520203e29e3,New Kam Man,200 Canal St,40.716973,-73.998302,"[{'label': 'display', 'lat': 40.71697342168782...",309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46802950,Canal St,4-6-6 Express
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,e-0-4b4cc100f964a52061bd26e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b4cc100f964a52061bd26e3,Key Food,1769 2nd Ave,40.781967,-73.949131,"[{'label': 'display', 'lat': 40.78196718550217...",306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96th St,Q
1,e-0-5490d200498e5cc0594a6e77-1,0,"[{'summary': 'This spot is popular', 'type': '...",5490d200498e5cc0594a6e77,Cherry Valley MarketPlace,1968 2nd Ave,40.787561,-73.944568,"[{'label': 'display', 'lat': 40.78756080289349...",425,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96th St,Q
2,e-0-4b256ab7f964a5209e7124e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b256ab7f964a5209e7124e3,Food Universe,1486 Lexington Ave,40.786084,-73.951041,"[{'label': 'display', 'lat': 40.786084, 'lng':...",393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96th St,Q
3,e-0-5be1cdb8e97dfb002ced1b25-3,0,"[{'summary': 'This spot is popular', 'type': '...",5be1cdb8e97dfb002ced1b25,Westside Market,Lexington avenue,40.783398,-73.952721,"[{'label': 'display', 'lat': 40.78339760879645...",485,...,1425660,https://www.seamless.com/menu/westside-market-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN,96th St,Q


In [15]:
dfs

,Unnamed: 0,Station_Name,LINE,lat,lng,numuni,num_super
0,0,Astor Pl,4-6-6 Express,40.730054,-73.991070,27,4
1,1,Canal St,4-6-6 Express,40.718814,-74.001055,17,8
2,2,50th St,1-2,40.762456,-73.985984,46,4
3,3,Bergen St,2-3-4,40.686111,-73.990756,30,2
4,4,Pennsylvania Ave,3-4,40.664714,-73.894886,0,3
...,...,...,...,...,...,...,...
468,468,Coney Island - Stillwell Av,D-F-N-Q,40.577281,-73.981236,0,0
469,469,34th St - Hudson Yards,7-7 Express,40.755446,-74.002197,3,1
470,470,72nd St,Q,40.768803,-73.958362,36,2
471,471,86th St,Q,40.777861,-73.951771,37,5


In [16]:
len(super_venues)

16

In [17]:
super_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name,venue.location.neighborhood,venue.venuePage.id,Station,LINE
0,e-0-4a98599ef964a520a62b20e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4a98599ef964a520a62b20e3,Sunrise Mart,4 Stuyvesant St,40.730150,-73.989448,"[{'label': 'display', 'lat': 40.73014952006944...",137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95825975,Astor Pl,4-6-6 Express
1,e-0-441a8599f964a52036311fe3-1,0,"[{'summary': 'This spot is popular', 'type': '...",441a8599f964a52036311fe3,Trader Joe's,142 E 14th St,40.733515,-73.987722,"[{'label': 'display', 'lat': 40.73351486923294...",477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Astor Pl,4-6-6 Express
2,e-0-4b8d2319f964a520c8e932e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b8d2319f964a520c8e932e3,Gristedes Supermarkets,25 University Pl,40.731977,-73.994553,"[{'label': 'display', 'lat': 40.73197675660857...",363,...,1007888,https://www.seamless.com/menu/gristedes-superm...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN,Astor Pl,4-6-6 Express
3,e-0-45b3d761f964a520b1411fe3-3,0,"[{'summary': 'This spot is popular', 'type': '...",45b3d761f964a520b1411fe3,Food Emporium,10 Union Sq E,40.734341,-73.989531,"[{'label': 'display', 'lat': 40.73434079522086...",494,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Astor Pl,4-6-6 Express
0,e-0-4b5cae76f964a520203e29e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b5cae76f964a520203e29e3,New Kam Man,200 Canal St,40.716973,-73.998302,"[{'label': 'display', 'lat': 40.71697342168782...",309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46802950,Canal St,4-6-6 Express
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,e-0-4b4cc100f964a52061bd26e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b4cc100f964a52061bd26e3,Key Food,1769 2nd Ave,40.781967,-73.949131,"[{'label': 'display', 'lat': 40.78196718550217...",306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96th St,Q
1,e-0-5490d200498e5cc0594a6e77-1,0,"[{'summary': 'This spot is popular', 'type': '...",5490d200498e5cc0594a6e77,Cherry Valley MarketPlace,1968 2nd Ave,40.787561,-73.944568,"[{'label': 'display', 'lat': 40.78756080289349...",425,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96th St,Q
2,e-0-4b256ab7f964a5209e7124e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b256ab7f964a5209e7124e3,Food Universe,1486 Lexington Ave,40.786084,-73.951041,"[{'label': 'display', 'lat': 40.786084, 'lng':...",393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96th St,Q
3,e-0-5be1cdb8e97dfb002ced1b25-3,0,"[{'summary': 'This spot is popular', 'type': '...",5be1cdb8e97dfb002ced1b25,Westside Market,Lexington avenue,40.783398,-73.952721,"[{'label': 'display', 'lat': 40.78339760879645...",485,...,1425660,https://www.seamless.com/menu/westside-market-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN,96th St,Q


In [18]:
super_venues.to_csv('super_venues.csv')

In [19]:
dfs.to_csv('dfs.csv')